In [1]:
!pip install tensorflow-gpu
!pip install pymatgen mendeleev graphviz pydot sklearn
!pip install crysnet
import tensorflow as tf
print(tf.test.gpu_device_name())
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 511.7 MB 6.3 kB/s 
     |████████████████████████████████| 438 kB 66.7 MB/s 
     |████████████████████████████████| 1.6 MB 52.6 MB/s 
     |████████████████████████████████| 5.8 MB 68.9 MB/s 
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.8.0
    Uninstalling tensorflow-estimator-2.8.0:
      Successfully uninstalled tensorflow-estimator-2.8.0
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.8.0
    Uninstalling tensorboard-2.8.0:
      Successfully uninstalled tensorboard-2.8.0
  Attempting uninstall: keras
    Found existing installation: keras 2.8.0
    Uninstalling keras-2.8.0:
      Successfully uninstalled keras-2.8.0
  Attempting uninstall: gast
    Found existing installation: gast 0.5.3
    Uninstalling gast-0.5.3:
      Successfully uninstalled gast-0.

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 2.7 MB 32.8 MB/s 
  Created wheel for crysnet: filename=crysnet-0.2.8-py3-none-any.whl size=2744649 sha256=c2de2b0d8b912a54aaddebf42c7b4e330699aa6218a3b41e7d1b7543b7bb5652
  Stored in directory: /root/.cache/pip/wheels/4c/91/dc/2a35b393d2a8192e3283d9073d4d003818bb05a7c1b0e94e25
Successfully built crysnet
/device:GPU:0
Num GPUs Available:  1


In [2]:
from pathlib import Path
from crysnet.data import Dataset
from crysnet.data.generator import GraphGenerator
from crysnet.models import GNN
import os
from pymatgen.core.structure import Structure
from crysnet.models import GNN
from crysnet.models.gnnmodel import (
    MpnnBaseModel, TransformerBaseModel , DirectionalMpnnModel, 
    DirectionalTransformerModel, MpnnModel, TransformerModel, 
    GraphModel, CgcnnModel, GraphAttentionModel
)
from keras.models import Model
from keras import layers
import tensorflow as tf
from tensorflow.keras import datasets, layers, models
import matplotlib.pyplot as plt
from crysnet.layers import MessagePassing
from crysnet.layers import PartitionPadding
from crysnet.layers import EdgeMessagePassing
import pandas as pd
import numpy as np
import re


Extract the feature vectors for nodes and edges.
NODES-number of connections of a node: found by calculating the number of times
a node is listed in the bar connectivity columns
Bar connectivity for the first entry is -



```
Node: connectivity(number of times it appears)
1: 3
2: 3
3: 3
4: 3
5: 3
6: 3
7: 3
8: 3 all the nodes from 1 to 8 appear 3 times
```




In [4]:
df  = pd.read_csv("unit_cell_catalog2000.csv")

In [5]:
bar_connectivity = list(filter(None,[re.findall(r'^bar_connectivity\w+', i) for i in df.columns]))

In [6]:
nodal_positions = list(filter(None,[re.findall(r'^nodal_positions\w+', i) for i in df.columns]))

In [7]:
d = np.sqrt((df[nodal_positions[3][0]]-df[nodal_positions[0][0]])**2)+\
np.sqrt((df[nodal_positions[4][0]]-df[nodal_positions[1][0]])**2)+\
np.sqrt((df[nodal_positions[5][0]]-df[nodal_positions[2][0]])**2)
d

0       1.00000
1       1.00000
2       1.00000
3       1.00000
4       1.00000
         ...   
1995    0.50000
1996    1.24085
1997    0.50000
1998    1.00000
1999    1.22326
Length: 2000, dtype: float64

In [8]:
l = (df[nodal_positions[3][0]]-df[nodal_positions[0][0]])/d
m = (df[nodal_positions[4][0]]-df[nodal_positions[1][0]])/d
n = (df[nodal_positions[5][0]]-df[nodal_positions[2][0]])/d

In [16]:
new_df = pd.DataFrame()
new_df['a'] = df['a']
new_df['b'] = df['b']
new_df['c'] = df['c']
new_df['alpha'] = df['alpha']
new_df['beta'] = df['beta']
new_df['gamma'] = df['gamma']
new_df['node_connectivity'] = df[bar_connectivity[0][0]]
new_df['d'] = d
new_df['l'] = l
new_df['m'] = m
new_df['n'] = n
new_df['Cx'] = df['Cx']
new_df['Cy'] = df['Cy']
new_df['Cz'] = df['Cz']
new_df['nx'] = df['nx']
new_df['ny'] = df['ny']
new_df['nz'] = df['nz']
new_df


a        b        c  alpha  beta  gamma  node_connectivity  \
0     1.00000  1.00000  1.00000     90  90.0     90                  1   
1     0.85075  0.57449  1.00000     90  90.0     90                  1   
2     1.00000  1.00000  1.00000     90  90.0     90                  1   
3     0.99471  0.99471  1.00000     90  90.0    120                  1   
4     0.50300  0.50300  1.00000     90  90.0     90                  1   
...       ...      ...      ...    ...   ...    ...                ...   
1995  0.68452  1.00000  0.34059     90  90.0     90                  1   
1996  0.99994  1.00000  0.80392     90  90.2     90                  9   
1997  0.38120  0.44449  1.00000     90  90.0     90                  2   
1998  0.22690  0.22690  1.00000     90  90.0     90                  1   
1999  1.00000  0.97372  0.72121     90  99.4     90                  1   

            d         l         m     n    Cx    Cy    Cz    nx    ny    nz  
0     1.00000  0.000000  1.000000  2000  0.33  0.33  0.33  1.00  1.00  1.00  
1     1.00000  0.000000  1.000000  2000  0.23  0.03  0.26  1.01  1.01  1.00  
2     1.00000  0.000000  1.000000  2000  0.09  0.09  0.09  2.00  2.00  2.00  
3     1.00000  0.000000  1.000000  2000  0.25  0.25  0.25  1.00  1.00  1.00  
4     1.00000  0.000000  1.000000  2000  0.25  0.25  0.50  1.00  1.00  1.00  
...       ...       ...       ...   ...   ...   ...   ...   ...   ...   ...  
1995  0.50000  0.000000  1.000000  2000  0.41  0.16  0.25  1.22  1.83  1.05  
1996  1.24085 -0.201475  0.597050  2000  0.15  0.15  0.28  1.99  1.98  1.04  
1997  0.50000  0.000000  1.000000  2000  0.24  0.13  0.09  1.01  1.01  1.01  
1998  1.00000  0.000000  1.000000  2000  0.01  0.01  0.26  1.05  1.05  1.00  
1999  1.22326 -0.204372 -0.591256  2000  0.13  0.12  0.36  2.00  2.00  1.35  

[2000 rows x 17 columns]

In [79]:
n = len(new_df)
train_df = new_df[0:int(n*0.8)]
val_df = new_df[int(n*0.8):int(n*0.9)]
test_df = new_df[int(n*0.9):]

In [81]:
print(train_df.shape,val_df.shape,test_df.shape)

(1600, 17) (200, 17) (200, 17)


<tf.Tensor: shape=(1600, 11), dtype=float64, numpy=
array([[ 1.00000000e+00,  1.00000000e+00,  1.00000000e+00, ...,
         0.00000000e+00,  1.00000000e+00,  2.00000000e+03],
       [ 8.50750000e-01,  5.74490000e-01,  1.00000000e+00, ...,
         0.00000000e+00,  1.00000000e+00,  2.00000000e+03],
       [ 1.00000000e+00,  1.00000000e+00,  1.00000000e+00, ...,
         0.00000000e+00,  1.00000000e+00,  2.00000000e+03],
       ...,
       [ 9.54470000e-01,  9.54470000e-01,  1.00000000e+00, ...,
         0.00000000e+00,  1.00000000e+00,  2.00000000e+03],
       [ 1.00000000e+00,  9.20910000e-01,  8.23460000e-01, ...,
         0.00000000e+00, -5.14958674e-01,  2.00000000e+03],
       [ 1.00000000e+00,  9.24210000e-01,  9.88470000e-01, ...,
        -2.83071209e-01, -7.16928791e-01,  2.00000000e+03]])>

In [139]:
X_train = train_df[
          ['a','b','c','alpha',
          'beta','gamma','node_connectivity',
          'd','l','m','n']
          ].to_numpy()
# y_train = train_df[['Cx','Cy','Cz','nx','ny','nz']].to_numpy()
y_train = train_df[['Cx']].to_numpy()

X_val = val_df[
          ['a','b','c','alpha',
          'beta','gamma','node_connectivity',
          'd','l','m','n']
          ].to_numpy()
y_val = val_df[['Cx','Cy','Cz','nx','ny','nz']].to_numpy()

In [140]:
normalizer = tf.keras.layers.Normalization(axis=-1)
normalizer.adapt(X_train)

In [141]:
model = tf.keras.Sequential([
    normalizer,
    tf.keras.layers.Dense(10, activation='relu'),
    tf.keras.layers.Dense(10, activation='relu'),
    tf.keras.layers.Dense(1)
  ])

# model.summary()

In [142]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

history = model.fit(X_train, y_train, epochs=10,batch_size=2)

Epoch 1/10
800/800 [==============================] - 4s 2ms/step - loss: nan - accuracy: 6.2500e-04
Epoch 2/10
800/800 [==============================] - 2s 2ms/step - loss: nan - accuracy: 6.2500e-04
Epoch 3/10
800/800 [==============================] - 2s 2ms/step - loss: nan - accuracy: 6.2500e-04
Epoch 4/10
800/800 [==============================] - 2s 2ms/step - loss: nan - accuracy: 6.2500e-04
Epoch 5/10
800/800 [==============================] - 2s 2ms/step - loss: nan - accuracy: 6.2500e-04
Epoch 6/10
800/800 [==============================] - 4s 5ms/step - loss: nan - accuracy: 6.2500e-04
Epoch 7/10
800/800 [==============================] - 3s 3ms/step - loss: nan - accuracy: 6.2500e-04
Epoch 8/10
800/800 [==============================] - 2s 2ms/step - loss: nan - accuracy: 6.2500e-04
Epoch 9/10
800/800 [==============================] - 2s 2ms/step - loss: nan - accuracy: 6.2500e-04
Epoch 10/10
800/800 [==============================] - 2s 2ms/step - loss: nan - accuracy: 